# Pseudo-code for project 2

OBJECTIVE: Find parameters of shots that lead to goals using exhaustive parameter scan

Should be at least 3 pages long

First, define shooting alogthim

Second, run shooting alogthim through raange of paramters. 

Objectives
 1. talk about drag and lift coeffcients of ball
 2. simulate trajectories of free kick and corner kick
 3. plot in 2D and 3D and desricbe paramters of trajectories
 4. Talk in depth about at one goal scoring shot for each: include parameters plots and a plot where omega = 0 to show influence of spin.

trajectory of free kick should stop calculating when
  1. ball reaches goal line
  2. ball touches ground
  3. ball touches wall

trajectory of corner kick should stop calculating when
  1. ball reaches goal line
  2. ball touches ground

Possbily use 3D-plotting to show trajectories of shots. (especally goal-making shots).

include refences and acknolegdments 
